In [2]:
# Imports


In [15]:
CONNECTION_PARAMETERS = {
    "account": "ug94937.us-east4.gcp",
    "user":"ADITYASINGH",
    "password": os.environ.get('SF_Password'),
    "role": "ADITYASINGH",
    "database": "FIRST_DB",
    "warehouse": "FOSFOR_INSIGHT_WH",
    "schema": "PUBLIC"
}

In [26]:
def create_stage(session, stage_name="demo"):
    try:
        session.sql(f"create or replace stage {stage_name}").collect()
        return f"@{stage_name}"
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex

        
def get_session():
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex


# Stored Procedure 
def run_experiment(session: Session, exp_data: str) -> list:
    # Imports
    from snowflake.snowpark.session import Session
    from snowflake.ml.modeling.pipeline import Pipeline
    from snowflake.ml.modeling.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
    from snowflake.ml.modeling.metrics import r2_score, accuracy_score, precision_score, roc_auc_score, \
        f1_score, recall_score, log_loss
    from snowflake.snowpark.functions import col, is_null, regexp_replace, when, lit
    from snowflake.snowpark.types import StringType
    from snowflake.snowpark.exceptions import SnowparkSQLException
    import importlib, sys, json, os, logging
    from snowflake.ml.registry.registry import Registry
    
    
    # Functions used in stored proc
    def apply_data_cleansing(df):
        """
        Method handles null values in snowpark dataframe.
        :param:
        df: input dataframe
        :returns:
        df_cleaned: dataframe after null handling
        """
        # fillna - Unknown and 0
        schema_fields = df.schema.fields
        fill_values = {field.name: "UNKNOWN" if isinstance(field.datatype, StringType) else 0 for field in schema_fields}
        df_cleaned = df.fillna(fill_values)
        return df_cleaned


    def get_feature_columns(df):
        """
        Identifies the numerical and categorical features in dataset.
        Identifies features for label encoding and one hot encoding
        :param:
        df: input dataframe
        :returns:
        categorical_features: list of non-numerical feature columns
        numerical_features: list of numerical feature columns
        le_column_features: list of feature label encoder columns
        oh_column_features: list of feature one hot encoder columns
        """
        schema_fields = df.schema.fields
        features = df.columns
        features.remove(exp_details.get("target_column"))
        df_schema = session.sql(f"DESCRIBE TABLE {exp_details.get('dataset')}").collect()
        categorical_types = ['VARCHAR','CHAR','STRING','TEXT','BOOL']
        categorical_features = []  
        for row in df_schema:
            for typ in categorical_types:
                if typ in row['type']:
                    categorical_features.append(row['name'])
                    break
        numerical_features = list(set(features) - set(categorical_features))
        print(f"numerical_features:  {numerical_features}")
        print(f"categorical_features: {categorical_features}")
        
        #identify columns for labelencoding and onehotencoding   
        le_column_features = []
        oh_column_features = []
        if len(categorical_features) >= 1:
            print(f"{categorical_features} columns are non numeric in feature dataset, encoding required.")
            for column in categorical_features:
                if df.select(df[column]).distinct().count() >= 10:
                    le_column_features.append(column)
                elif column == exp_details.get("target_column"):
                    le_column_feature.append(column)
                else:
                    oh_column_features.append(column)
            print(f"Columns identified to be encoded with label encoder: {le_column_features}")
            print(f"Columns identified to be encoded with one hot encoder: {oh_column_features}")
        return categorical_features, numerical_features, le_column_features, oh_column_features


    def create_and_run_preprocessing(df, categorical_features, numerical_features, le_column_features, oh_column_features):
        """
        Based on different features column input creates preprocessing steps and run it on input dataframe
        :param:
        df: input dataframe
        categorical_features: list of non-numerical feature columns
        numerical_features: list of numerical feature columns
        le_column_features: list of feature label encoder columns
        oh_column_features: list of feature one hot encoder columns
        :returns:
        df_train: preprocessed train dataset
        df_test: preprocessed test dataset
        """
        #pipeline steps 
        print("Setting up preprocessing pipeline based on dataset")
        categorical_pp = {f'le_{column}':LabelEncoder(input_cols=column, output_cols=column) for column in le_column_features}
        if len(oh_column_features)>0:
            categorical_pp['oh_enc'] = OneHotEncoder(input_cols=oh_column_features, output_cols=oh_column_features, handle_unknown='ignore')
        numerical_pp = {
            'scaler': MinMaxScaler(input_cols=numerical_features, output_cols=numerical_features)
        }
        steps = [(key, categorical_pp[key]) for key in categorical_pp if categorical_pp[key]!=[]] + \
        [(key, numerical_pp[key]) for key in numerical_pp if numerical_features!=[]]
        print(f"Selected preprocesing steps: \n{steps}")    
            
        # Run preprocessing pipeline steps 
        print("Running data preprocessing pipeline")
        df = Pipeline(steps=steps).fit(df).transform(df)
        print(f"Transformed dataset: \n {df.show()}")
        df_train, df_test = df.random_split(weights=[0.8, 0.2], seed=0)
        return df_train, df_test


    def run_estimator(df_train, df_test, input_cols):
        """
        trains on df_train and creates model object for given algorithm/estimator.
        runs prediction function of model object on test dataset
        :param:
        df_train: input training dataframe
        df_test: input test dataframe
        input_cols: list of input feature names
        :returns:
        df_pred: output predict dataframe
        """
        for algorithm, hyperparam in exp_details.get("algo_details").items():
            algorithm = algorithm.rsplit('.', 1)
            module = importlib.import_module(algorithm[0])
            print(f"----Running Algorithm {algorithm[1]}----")
            attr = getattr(module, algorithm[1])
            pipe = Pipeline(steps=[("algorithm", attr(input_cols=input_cols
                                                  , label_cols=[exp_details.get("target_column")]
                                                  , output_cols=['PREDICTIONS']))]
                   )
    
            # Fit the pipeline
            print(f"Running model pipeline {algorithm[1]}")
            model = pipe.fit(df_train)
     
            # Test the model
            print("Running prediction on model with test dataset")
            df_pred = model.predict(df_test)
            return model, df_pred

    
    def try_or(fn):
        try:
            out = fn()
            return out
        except:
            return None
    
        
    def eval_metrics(df_pred):
        print("Generating Metrices")
        return {
            'accuracy': try_or(lambda: accuracy_score(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names='PREDICTIONS')),
            'f1_score': try_or(lambda: f1_score(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names='PREDICTIONS')),
            'recall': try_or(lambda: recall_score(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names='PREDICTIONS')),
            'precision': try_or(lambda: precision_score(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names='PREDICTIONS')),
            'roc_auc': try_or(lambda: roc_auc_score(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_score_col_names='PREDICTIONS')),
            'log_loss': try_or(lambda: log_loss(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_score_col_names='PREDICTIONS'))
        }


    def register_model(model, metrics_info):
        print("Started: Registering model on snowflake")
        reg = Registry(session=session)
        try:
            model_name = exp_exp_details.get("project_id")+"_"+exp_details.get("id")+"_"+exp_details.get("run_id")
            mv = reg.log_model(model=model,
                               model_name=model_name,
                               comment=exp_details.get("description"),
                               version_name="V1",
                               python_version="3.9.19",
                               conda_dependencies=["xgboost","scikit-learn==1.2.2"],
                               metrics=[{"model_metrics": metrics_info, 
                                         "project_id": exp_details.get("project_id"),
                                         "id": exp_details.get("id"),
                                         "run_id": exp_details.get("run_id"),
                                         "algorithm_type": exp_details.get("algorithm_type"),
                                         "algo_details": exp_details.get("algo_details"),
                                         "dataset": exp_details.get("dataset"),
                                         "target_column": exp_details.get("target_column"),
                                         "exp_name": exp_details.get("exp_name"),
                                         "source": "EXP"}])
            print("Registeration of model completed!!!")
            return model_name
        except Exception as ex:
            key = 'Processing aborted due to error 370001' 
            if key in str(ex):
                log_message("INFO","Registeration of model completed!!!")
                pass
            else:
                log_message("ERROR","Exception Occured while registering model")
                return str(ex).split('?')
            
            
    def create_tags(session, exp_details, metric_names):
        for key in exp_details.keys():
            tag = session.sql(f"CREATE TAG IF NOT EXISTS {key}")
            tag.show()
        for name in metric_names:
            tag = session.sql(f"CREATE TAG IF NOT EXISTS {name}")
            tag.show()

        
    def set_tags(session, m_name, exp_details, metric_info):
        for key, value in exp_details.items():
            value = str(value).replace("'","\"")
            tag = session.sql(f"ALTER MODEL IF EXISTS {m_name} SET TAG {key}='{value}'")
            tag.show()
        for key, value in metric_info.items():
            value = str(value).replace("'","\"")
            tag = session.sql(f"ALTER MODEL IF EXISTS {m_name} SET TAG {key}='{value}'")
            tag.show()
    
    # loading experiment details
    exp_details=json.loads(exp_data)
    
    metric_names=["accuracy_score","precision_score","recall_score","f1_score","roc_auc","log_loss"]
    
    # creating user tags if not exist
    create_tags(session, exp_details, metric_names)
    
    # Reading dataset
    print("Reading dataset features")
    data = session.table(exp_details.get("dataset"))
    
    # fillna
    data = apply_data_cleansing(data)
    
    # Identify feature columns
    categorical_features, numerical_features, le_column_features, oh_column_features = get_feature_columns(data)
    
    # Based on feature, do preprocessing
    data_train, data_test = create_and_run_preprocessing(data, categorical_features, numerical_features, le_column_features, oh_column_features)
    
    # Run model training and prediction
    input_cols = categorical_features+numerical_features
    if exp_details.get("target_column") in categorical_features:
        input_cols.remove(exp_details.get("target_column"))
    model, data_pred = run_estimator(data_train, data_test, input_cols)
    
    # Evaluate model metrices
    metrics_info = eval_metrics(data_pred)
    
    # Register model on snowflake registry
    model_name = register_model(model, metrics_info)
    
    # Set relevant tags to model object
    set_tags(session, model_name, exp_details, metric_info)
    
    return [{"exp_name":exp_details.get("exp_name", "sample_experiment"),
             "version":"V1",
             "matrices":{"model_metrics": metrics_info, "project_id": "0001", "source": "EXP"},
             "algorithm_type":exp_details.get("algorithm_type"),
             "algorithm": list(exp_details.get("algo_details").keys()),
             "RUN_STATUS":"SUCCESS",
             "registry_model_name":exp_details.get("project_id")+"_"+exp_details.get("id")+"_"+exp_details.get("run_id")}]

In [24]:
import uuid
# project_id = str(uuid.uuid4())
run_id = str(uuid.uuid4())
print(run_id)

3cc38fbf-f6e3-42f9-933b-5b9557fe649e


In [27]:
%%time
# Initilization
# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print("Creating Snowflake Session object...")
session = get_session()
stage = create_stage(session)
print("Session has been created !")

print("Creating stored procedure...")
session.sproc.register(func=run_experiment,
                       name="run_experiment",
                       packages=["snowflake-snowpark-python", "snowflake-ml-python"],
                       isPermanant=False,
                       stage_location=stage,
                       replace=True)
print("Stored procedure has been created successfully!")

# payload
exp_data = '{"source":"EXP", "project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", "id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", "run_id":"afb65a5a-082f-4f47-bc98-92de1d3a4055", "exp_name": "py_func_exp23", "algorithm_type":"classification", "algo_details": {"snowflake.ml.modeling.xgboost.XGBClassifier": null}, "dataset": "AIRLINE_DEP_DELAY_10K", "target_column": "DEP_DEL15"}'

# tagging session
print("Setting tag to session object", run_id)
session.query_tag='afb65a5a-082f-4f47-bc98-92de1d3a4055'

print("Executing Procedure")
procedure_response = session.call("run_experiment", exp_data)
# procedure_response = run_experiment(session, exp_data)
print("Stored Procedure Executed Successfully !")
print(procedure_response)

#Log in mlflow
print("Logging in mlflow completed !")

Creating Snowflake Session object...
Session has been created !
Creating stored procedure...


The version of package 'snowflake-snowpark-python' in the local environment is 1.18.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'snowflake-ml-python' in the local environment is 1.5.1, which does not fit the criteria for the requirement 'snowflake-ml-python'. Your UDF might not work when the package version is different between the server and your local environment.


Stored procedure has been created successfully!
Setting tag to session object 3cc38fbf-f6e3-42f9-933b-5b9557fe649e
Executing Procedure
-----------------------------------------------
|"status"                                     |
-----------------------------------------------
|SOURCE already exists, statement succeeded.  |
-----------------------------------------------

---------------------------------------------------
|"status"                                         |
---------------------------------------------------
|PROJECT_ID already exists, statement succeeded.  |
---------------------------------------------------

-------------------------------------------
|"status"                                 |
-------------------------------------------
|ID already exists, statement succeeded.  |
-------------------------------------------

-----------------------------------------------
|"status"                                     |
----------------------------------------------

----Running Algorithm XGBClassifier----
Running model pipeline XGBClassifier


The version of package 'snowflake-snowpark-python' in the local environment is 1.18.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.


Running prediction on model with test dataset
Generating Metrices


DataFrame.flatten() is deprecated since 0.7.0. Use `DataFrame.join_table_function()` instead.
The version of package 'snowflake-snowpark-python' in the local environment is 1.18.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.


Started: Registering model on snowflake


NameError: name 'log_message' is not defined